In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [2]:
train = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
test = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")

In [3]:
VOCAB_SIZE=1000
train = train.fillna('other', axis=0)
test = test.fillna('other', axis=0)

# Build text dataset
tfds = tf.data.Dataset.from_tensor_slices((list(train['text'].values), train['target'].values))
train_ds = tfds.take(5000)
val_ds = tfds.skip(5000)
train_ds = train_ds.shuffle(1000).batch(32).prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.batch(32).prefetch(tf.data.AUTOTUNE)

# Build keyword dataset
tfds_extra = tf.data.Dataset.from_tensor_slices((list(train['keyword'].values), train['target'].values))
train_extra = tfds_extra.take(5000)
val_extra = tfds_extra.skip(5000)
train_extra = train_extra.shuffle(1000).batch(32).prefetch(tf.data.AUTOTUNE)
val_extra = val_extra.shuffle(1000).batch(32).prefetch(tf.data.AUTOTUNE)

# Build encoder layer
encoder = tf.keras.layers.TextVectorization(max_tokens=VOCAB_SIZE)
encoder.adapt(tfds.map(lambda text, lab: text))

In [4]:
vocab = np.array(encoder.get_vocabulary())
vocab[0:20]

array(['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is', 'for',
       'on', 'you', 'my', 'with', 'it', 'that', 'at', 'by', 'this'],
      dtype='<U17')

In [5]:
def generator():
    for i in range(len(test)):
        yield {'text': test['text'][i], 'keyword': test['keyword'][i]}

In [6]:
text_input = tf.keras.Input(shape=(1,), dtype=tf.string)
text_model = encoder(text_input)
text_model = tf.keras.layers.Embedding(
        input_dim=len(vocab),
        output_dim=64,
        mask_zero=True)(text_model)
text_model = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64))(text_model)
text_model = tf.keras.layers.Dense(64, activation='relu')(text_model)
text_model = tf.keras.layers.Dense(1)(text_model)

extra_input = tf.keras.Input(shape=(1,), dtype=tf.string)
extra_model = encoder(extra_input)
extra_model = tf.keras.layers.Embedding(
        input_dim=len(vocab),
        output_dim=64,
        input_length=1)(extra_model)
extra_model = tf.keras.layers.GlobalAveragePooling1D()(extra_model)
extra_model = tf.keras.layers.Dense(64, activation='relu')(extra_model)
extra_model = tf.keras.layers.Dense(1)(extra_model)

comb_model = tf.keras.layers.concatenate([text_model, extra_model])
comb_model = tf.keras.layers.Dense(1)(comb_model)
comb_model = tf.keras.Model(inputs=[text_input, extra_input], outputs=comb_model)

In [7]:

comb_model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])
comb_model.fit(x=[train['text'], train['keyword']], y=train['target'],
               epochs=10, validation_split=0.3, batch_size=32)

Epoch 1/10
167/167 [==============================] - 19s 54ms/step - loss: 0.6886 - accuracy: 0.5755 - val_loss: 0.6864 - val_accuracy: 0.5582
Epoch 2/10
167/167 [==============================] - 6s 36ms/step - loss: 0.6610 - accuracy: 0.5755 - val_loss: 0.6490 - val_accuracy: 0.5582
Epoch 3/10
167/167 [==============================] - 6s 33ms/step - loss: 0.5787 - accuracy: 0.6382 - val_loss: 0.5611 - val_accuracy: 0.7207
Epoch 4/10
167/167 [==============================] - 6s 34ms/step - loss: 0.4822 - accuracy: 0.7840 - val_loss: 0.5205 - val_accuracy: 0.7526
Epoch 5/10
167/167 [==============================] - 6s 33ms/step - loss: 0.4396 - accuracy: 0.8024 - val_loss: 0.5129 - val_accuracy: 0.7509
Epoch 6/10
167/167 [==============================] - 6s 34ms/step - loss: 0.4167 - accuracy: 0.8135 - val_loss: 0.5419 - val_accuracy: 0.7653
Epoch 7/10
167/167 [==============================] - 6s 39ms/step - loss: 0.3996 - accuracy: 0.8223 - val_loss: 0.5141 - val_accuracy: 0.754

In [8]:
targets = comb_model.predict(x=[test['text'], test['keyword']])
targets = [1 if target > 0 else 0 for target in targets]
test['target'] = targets
test[['id','target']].to_csv("/kaggle/working/submission.csv", index=False)

102/102 [==============================] - 4s 10ms/step
